In [9]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential

In [10]:
data_dir = r'Train'

In [11]:
categories = os.listdir(data_dir)

In [12]:
IMG_SIZE = 32
CHANNELS = 3

In [13]:
def preprocess_data():
    data = []
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([img_array, class_num])
            except Exception as e:
                print(f"Error processing image: {e}")
    return data

In [14]:
data = preprocess_data()

In [15]:
np.random.shuffle(data)

In [16]:
X = np.array([entry[0] for entry in data])
y = np.array([entry[1] for entry in data])

In [17]:
X = X / 255.0

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, CHANNELS)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(categories), activation='softmax'))


c:\Users\iceki\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1372 - loss: 2.0869 - val_accuracy: 0.1710 - val_loss: 2.0647
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2019 - loss: 2.0374 - val_accuracy: 0.2187 - val_loss: 1.9465
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2579 - loss: 1.9222 - val_accuracy: 0.2783 - val_loss: 1.8498
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3667 - loss: 1.7407 - val_accuracy: 0.4473 - val_loss: 1.5681
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5024 - loss: 1.4347 - val_accuracy: 0.5030 - val_loss: 1.4298
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6021 - loss: 1.1997 - val_accuracy: 0.5229 - val_loss: 1.4337
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6444 - loss: 1.0456 - val_accuracy: 0.5646 - val_loss: 1.3030
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7196 - loss: 0.8668 - val_accuracy: 0.5964 - val_loss:

In [22]:
_, test_acc = model.evaluate(x_test,y_test)
print(test_acc)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6509 - loss: 2.3078 
0.6361829042434692


In [23]:
def preprocess_image(image_path):
    img_array = cv2.imread(image_path)
    img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    img_array = img_array / 255.0
    return img_array.reshape(-1, IMG_SIZE, IMG_SIZE, CHANNELS)

In [24]:
def classify_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    prediction = model.predict(preprocessed_image)
    predicted_class_index = np.argmax(prediction)
    predicted_class = categories[predicted_class_index]
    return predicted_class

In [25]:
test_image_path = r'Toyota.jpg'

In [26]:
predicted_class = classify_image(test_image_path)
print("Predicted class:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Predicted class: toyota
